# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import sqlite3
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [3]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""SELECT customerNumber, contactLastName, contactFirstName
                    FROM customers
                    WHERE customerNumber IN 
                        (SELECT customerNumber 
                                FROM orders
                                WHERE orderDate = '2003-01-31')""")
c.fetchall()

[('141', 'Freyre', 'Diego ')]

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [33]:
# for each product name, sum up all the quantities ordered of that product name
# HOW TF R U SUPPOSED TO DO THIS WITH SUBQUERIES????
c.execute("""SELECT SUM(quantityOrdered) AS total_number_of_orders, productName 
                    FROM orderdetails
                    JOIN products USING(productCode)
                    GROUP BY productName
                    ORDER BY total_number_of_orders DESC""")
c.fetchall()

[(1808, '1992 Ferrari 360 Spider red'),
 (1111, '1937 Lincoln Berline'),
 (1085, 'American Airlines: MD-11S'),
 (1076, '1941 Chevrolet Special Deluxe Cabriolet'),
 (1074, '1930 Buick Marquette Phaeton'),
 (1061, '1940s Ford truck'),
 (1057, '1969 Harley Davidson Ultimate Chopper'),
 (1056, '1957 Chevy Pickup'),
 (1053, '1964 Mercedes Tour Bus'),
 (1052, '1956 Porsche 356A Coupe'),
 (1051, 'Corsair F4U ( Bird Cage)'),
 (1047, 'F/A 18 Hornet 1/72'),
 (1040, '1980s Black Hawk Helicopter'),
 (1038, '1913 Ford Model T Speedster'),
 (1033, '1997 BMW R 1100 S'),
 (1030, '1972 Alfa Romeo GTA'),
 (1029, '1962 Volkswagen Microbus'),
 (1028, '2002 Suzuki XREO'),
 (1020, 'The USS Constitution Ship'),
 (1019, '2001 Ferrari Enzo'),
 (1015, '1960 BSA Gold Star DBD34'),
 (1014, '1997 BMW F650 ST'),
 (1013, '1957 Corvette Convertible'),
 (1011, '18th century schooner'),
 (1009, '1900s Vintage Tri-Plane'),
 (999, '1940 Ford Pickup Truck'),
 (999, '1996 Moto Guzzi 1100i'),
 (998, '1926 Ford Fire Engine')

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [22]:
# c.execute("""SELECT COUNT(DISTINCT c.customerNumber) total_num_of_people, p.productName
#                 FROM customers c
#                 JOIN orders USING('customerNumber')
#                 JOIN orderdetails USING ('orderNumber')
#                 JOIN products p USING('productCode')
#                 GROUP BY p.productName
#                 ORDER BY total_num_of_people DESC""")
# c.fetchall()

#BELOW: This gets all products that've been sold by < 20 people
c.execute("""SELECT e.employeeNumber, COUNT(DISTINCT c.customerNumber) total_num_of_people, p.productName
                            FROM employees e
                            JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
                            JOIN orders USING('customerNumber')
                            JOIN orderdetails USING ('orderNumber')
                            JOIN products p USING('productCode')
                            GROUP BY p.productName
                            HAVING total_num_of_people < 20""")
c.fetchall()

[('1165', 18, '1949 Jaguar XK 120'),
 ('1165', 18, '1952 Citroen-15CV'),
 ('1165', 19, '1958 Chevy Corvette Limited Edition'),
 ('1165', 18, '1969 Chevrolet Camaro Z28'),
 ('1165', 18, '2002 Chevy Corvette')]

## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [58]:
# c.execute("""SELECT DISTINCT e.employeeNumber, o.officeCode, o.city, e.firstName, e.lastName
#                     FROM employees AS e
#                     JOIN offices AS o USING(officeCode)
#                     WHERE e.employeeNumber IN 
#                     (SELECT e.employeeNumber, COUNT(DISTINCT c.customerNumber) AS total_num_of_people, 
#                             p.productName
#                             FROM employees AS e
#                             JOIN customers AS c ON e.employeeNumber = c.salesRepEmployeeNumber
#                             JOIN orders USING('customerNumber')
#                             JOIN orderdetails USING ('orderNumber')
#                             JOIN products AS p USING('productCode')
#                             GROUP BY p.productName
#                             HAVING total_num_of_people < 20
#                     )
                    
#             """)
# c.fetchall()

c.execute("""SELECT DISTINCT e.employeeNumber, o.officeCode, o.city, e.firstName, e.lastName
                    FROM employees e
                    JOIN offices o USING(officeCode)
                    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
                    JOIN orders USING (customerNumber)
                    JOIN orderdetails USING (orderNumber)
                    WHERE productCode IN 
                    (SELECT productCode
                            FROM products 
                            JOIN orderdetails USING(productCode)
                            JOIN orders USING(orderNumber)
                            JOIN customers USING(customerNumber)
                            JOIN employees ON salesRepEmployeeNumber = employeeNumber
                            GROUP BY productCode
                            HAVING COUNT(DISTINCT customerNumber) < 20
                    )

            """)
c.fetchall()

[('1370', '4', 'Paris', 'Gerard', 'Hernandez'),
 ('1501', '7', 'London', 'Larry', 'Bott'),
 ('1337', '4', 'Paris', 'Loui', 'Bondur'),
 ('1166', '1', 'San Francisco', 'Leslie', 'Thompson'),
 ('1286', '3', 'NYC', 'Foon Yue', 'Tseng'),
 ('1612', '6', 'Sydney', 'Peter', 'Marsh'),
 ('1611', '6', 'Sydney', 'Andy', 'Fixter'),
 ('1401', '4', 'Paris', 'Pamela', 'Castillo'),
 ('1621', '5', 'Tokyo', 'Mami', 'Nishi'),
 ('1323', '3', 'NYC', 'George', 'Vanauf'),
 ('1165', '1', 'San Francisco', 'Leslie', 'Jennings'),
 ('1702', '4', 'Paris', 'Martin', 'Gerard'),
 ('1216', '2', 'Boston', 'Steve', 'Patterson'),
 ('1188', '2', 'Boston', 'Julie', 'Firrelli'),
 ('1504', '7', 'London', 'Barry', 'Jones')]

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [64]:
c.execute("""SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(customerNumber)
                    FROM employees e
                    JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
                    WHERE c.creditLimit > 15000
                    GROUP BY e.employeeNumber
                    ORDER BY e.employeeNumber
          """)

c.fetchall()

[('1165', 'Leslie', 'Jennings', 6),
 ('1166', 'Leslie', 'Thompson', 6),
 ('1188', 'Julie', 'Firrelli', 6),
 ('1216', 'Steve', 'Patterson', 6),
 ('1286', 'Foon Yue', 'Tseng', 7),
 ('1323', 'George', 'Vanauf', 8),
 ('1337', 'Loui', 'Bondur', 6),
 ('1370', 'Gerard', 'Hernandez', 7),
 ('1401', 'Pamela', 'Castillo', 10),
 ('1501', 'Larry', 'Bott', 8),
 ('1504', 'Barry', 'Jones', 9),
 ('1611', 'Andy', 'Fixter', 5),
 ('1612', 'Peter', 'Marsh', 5),
 ('1621', 'Mami', 'Nishi', 5),
 ('1702', 'Martin', 'Gerard', 6)]

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!